# JB2008 Model Comparisons - jb08 (using Pyatmos) vs CCMC

In [1]:
import sys, os
import requests
import pandas as pd
import numpy as np
import netCDF4 as nc

jb_path = r'D:\GitHub\jb2008\jb08'
sys.path.append(os.path.dirname(jb_path))


In [2]:
import jb08

Running Setup:
Checking for and downloading required files.
...


In [3]:
http_f = r'D:\JB08.2002002.nc'
dat = nc.Dataset(http_f) 
#with requests.get(http_f,stream=True) as r:
#  dat = nc.Dataset('in-mem-file', mode='r', memory=r.content) 

In [4]:
dat.variables

{'time': <class 'netCDF4._netCDF4.Variable'>
 float64 time(time)
     long_name: Time
     units: minutes
     axis: T
 unlimited dimensions: time
 current shape = (96,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'ht': <class 'netCDF4._netCDF4.Variable'>
 float32 ht(ht)
     long_name: Height
     units: km
     axis: Z
 unlimited dimensions: 
 current shape = (46,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'lat': <class 'netCDF4._netCDF4.Variable'>
 float32 lat(lat)
     long_name: Latitude
     units: degrees_north
     axis: Y
 unlimited dimensions: 
 current shape = (37,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'lon': <class 'netCDF4._netCDF4.Variable'>
 float32 lon(lon)
     long_name: Longitude
     units: degrees_east
     axis: X
 unlimited dimensions: 
 current shape = (73,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'Temp_exo': <class 'netCDF4._netCDF4.Variable'>
 float32 Temp_exo(time, la

In [5]:
t = pd.to_datetime('2002-01-02')
dt = [pd.to_datetime('2002-01-02')+pd.Timedelta(v, unit='minutes') for v in np.array(dat.variables['time'][:])]
lat = np.array(dat.variables['lat'][::2])
lon = np.array(dat.variables['lon'][::2])
alt = np.array(dat.variables['ht'][::2])
jb_dens = np.array(dat.variables['DEN'][:,::2,::2,::2])

In [6]:
jb_swft = jb08.jb2008(t=dt,lat=lat,lon=lon,alt=alt,dtype='float64')
jb_swft.predict()

Dask Apply:   0%|          | 0/65 [00:00<?, ?it/s]

In [7]:
jb_swft.dat.shape

(1552224, 20)

In [8]:
print(jb_dens[0])
print(jb_swft.dat['DEN'][0])


[[[4.0726741e-07 4.0726741e-07 4.0726741e-07 ... 4.0726741e-07
   4.0726741e-07 4.0726741e-07]
  [4.1094552e-07 4.1095123e-07 4.1095464e-07 ... 4.1092625e-07
   4.1093722e-07 4.1094552e-07]
  [4.2151692e-07 4.2152891e-07 4.2153610e-07 ... 4.2147659e-07
   4.2149952e-07 4.2151692e-07]
  ...
  [7.0348881e-07 7.0350859e-07 7.0352041e-07 ... 7.0342219e-07
   7.0346010e-07 7.0348881e-07]
  [7.2123720e-07 7.2124720e-07 7.2125317e-07 ... 7.2120338e-07
   7.2122265e-07 7.2123720e-07]
  [7.2738050e-07 7.2738050e-07 7.2738050e-07 ... 7.2738050e-07
   7.2738050e-07 7.2738050e-07]]

 [[3.3641423e-09 3.3641974e-09 3.3642398e-09 ... 3.3643364e-09
   3.3642713e-09 3.3641423e-09]
  [3.3735561e-09 3.3726968e-09 3.3723031e-09 ... 3.3759953e-09
   3.3746048e-09 3.3735561e-09]
  [3.4282090e-09 3.4263452e-09 3.4254717e-09 ... 3.4330898e-09
   3.4302616e-09 3.4282090e-09]
  ...
  [5.1128413e-09 5.1099200e-09 5.1086237e-09 ... 5.1200808e-09
   5.1158642e-09 5.1128413e-09]
  [5.2379852e-09 5.2365743e-09 5.235

In [9]:
#(100*(jb_swft.dat['DEN']-jb_dens)/jb_swft.dat['DEN']).abs().describe()

In [10]:
jb_comp = np.swapaxes(jb_dens,0,1)
jb_comp = np.swapaxes(jb_comp,0,2)
jb_comp = np.swapaxes(jb_comp,2,3)
print(jb_dens.shape)
print(jb_comp.shape)

x, _, _, _ = np.meshgrid(dt,lat,lon,alt)
x.shape

(96, 23, 19, 37)
(19, 96, 37, 23)


(19, 96, 37, 23)

In [11]:
(100*(jb_swft.dat['DEN']-jb_comp.flatten())/jb_swft.dat['DEN']).abs().describe()

count    1.552224e+06
mean     1.248868e+00
std      1.802540e+00
min      0.000000e+00
25%      2.886324e-01
50%      7.252681e-01
75%      9.007875e-01
max      1.186699e+01
Name: DEN, dtype: float64

In [12]:
(100*(jb_swft.dat['DEN']-jb_comp.flatten())/jb_swft.dat['DEN']).abs().argmax()

900467

In [22]:
print(lat[11]) #11
print(lon[4]) #4
print(alt[-6])
print(dt[2])

20.0
40.0
780.0
2002-01-02 00:30:00


In [23]:
print(jb_swft.lat[900467])
print(jb_swft.lon[900467])
print(jb_swft.t[900467])
print(jb_swft.alt[900467])
print(jb_swft.dat['DEN'][900467])

20.0
40.0
2002-01-02T00:30:00.000000000
780.0
2.988855e-14


In [ ]:
jb_den

array([-90., -90., -90., ...,  90.,  90.,  90.])

In [16]:
print(jb_swft.dat[0:10]['DEN'])
print(jb_comp.flatten()[0:10])

0    4.072406e-07
1    3.367900e-09
2    6.703345e-10
3    2.284109e-10
4    9.428235e-11
5    4.402195e-11
6    2.220295e-11
7    1.184050e-11
8    6.588433e-12
9    3.790228e-12
Name: DEN, dtype: float32
[4.07267407e-07 3.36414230e-09 6.68375133e-10 2.26762886e-10
 9.31784996e-11 4.33117188e-11 2.17531045e-11 1.15552125e-11
 6.40601591e-12 3.67238011e-12]
